In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
df = pd.read_csv('KiteData_20231202_f4_flood.csv', delimiter= ";")

columns = df.columns

for column in columns:
    df[column] = df[column].str.replace(',', '.', regex=False).astype(float)

"""
df['time_group'] = df['time'].astype(int)

df_downsampled = df.groupby('time_group').mean(numeric_only=True).reset_index()
df_downsampled = df_downsampled.drop(columns = ['time'])

print(df_downsampled)"""



FileNotFoundError: [Errno 2] No such file or directory: 'Data/KiteData_20231202_f4_flood.csv'

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJS, RadioButtonGroup, Div
from bokeh.layouts import column
from bokeh.io import output_file


output_file("bokeh_plot.html")

df_downsampled_test = df.iloc[:, :]
LABELS = [c for c in df_downsampled_test.columns if c != 'time']


x_vals = df_downsampled_test['time'].to_list()
y_init = df_downsampled_test[LABELS[0]].to_list()

source = ColumnDataSource(data={'x': x_vals, 'y': y_init})

p = figure(title=f"{LABELS[0]} plot")
p.line('x', 'y', source=source)

features_title = Div(text="<b>Features</b>")
buttons = RadioButtonGroup(labels=LABELS, active=0)

all_series = {name: df_downsampled_test[name].to_list() for name in LABELS}


buttons.js_on_change('active',CustomJS(args=dict(
    src=source, series=all_series, labels=LABELS, fig=p
), code="""
    const idx = this.active;              // which button
    const label = labels[idx];            // feature name
    src.data['y'] = series[label];        // swap y data
    src.change.emit();                    // refresh plot
    fig.title.text = `${label} plot`; // update title
"""))


callback = CustomJS(args=dict(src=source, xr=p.x_range, yr=p.y_range), code="""
  const x = src.data['x'];
  const y = src.data['y'];
  const start = xr.start;
  const end = xr.end;

  let ymin = Infinity;
  let ymax = -Infinity;

  for (let i = 0; i < x.length; i++) {
    if (x[i] >= start && x[i] <= end) {
      if (y[i] < ymin) ymin = y[i];
      if (y[i] > ymax) ymax = y[i];
    }
  }

  if (ymin < Infinity) {
    const pad = (ymax - ymin) * 0.1 || 1;  // small padding
    yr.start = ymin - pad;
    yr.end = ymax + pad;
  }
""")
p.x_range.js_on_change('start', callback)
p.x_range.js_on_change('end', callback)


show(column(features_title, buttons, p, sizing_mode="stretch_width"))